Link https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

- Find the original problem description in kaggle here. SalePrice is the outcome variable. Use a clean version of the data that is treated for null values.
- Find number of categorical and continuous variables
- Some observations contain null values in SalesPrice. Drop those records from the analysis 
- Divide the data into training and test data 70/30 ratio with seed =1
- Build a model to estimate SalePrice excluding Id as feature. Calculate R2 and RMSE - Linear regression, Lasso, Ridge, Elastic net
- Take log of the sales price, does the R2 score improves?
- Try model with polynomial terms with degree = 2
- Try PCA - what is required number of principal components to retain 99% of variance?
- Try Feature selection - find 10 most significant features of the dataset. 


In [10]:
import pandas as pd
from sklearn import *
import numpy as np

In [8]:
df = pd.read_csv("/data/kaggle/data_combined_cleaned.csv")
df = df.dropna()
df = df.drop(columns = "Id")
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalesPrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,Gtl,...,0,None,None,None,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,Gtl,...,0,None,None,None,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,Gtl,...,0,None,None,None,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,Gtl,...,0,None,None,None,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,Gtl,...,0,None,None,None,0,12,2008,WD,Normal,250000.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 80 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non

In [12]:
target = "SalesPrice"

y = df[target]
y = np.log(y)
X = df.drop(columns=target)

X_dummy = pd.get_dummies(X, drop_first=True)


X_train, X_test, y_train, y_test = model_selection.train_test_split(X_dummy.values, y
                                        , test_size = 0.3, random_state = 1)

pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.LinearRegression())
])

pipe.fit(X_train, y_train)

est = pipe.steps[-1][-1]

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("r2 score on train", metrics.r2_score(y_train, y_train_pred))
print("r2 score on test", metrics.r2_score(y_test, y_test_pred))

print("rmse on train", np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("rmse score on test", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

r2 score on train 0.9494518297088844
r2 score on test -3.0715943247509893e+22
rmse on train 0.08606817747710854
rmse score on test 76190488351.17424


In [16]:
summary = pd.DataFrame({"feature": X_dummy.columns, "coeffient": est.coef_})
summary["mag"] = np.abs(summary.coeffient)
summary.sort_values("mag", ascending = False)

,feature,coeffient,mag
15,GrLivArea,1.501695e+11,1.501695e+11
13,2ndFlrSF,-1.233520e+11,1.233520e+11
12,1stFlrSF,-1.112376e+11,1.112376e+11
219,GarageFinish_None,-1.072428e+11,1.072428e+11
134,Exterior2nd_CBlock,8.844505e+10,8.844505e+10
120,Exterior1st_CBlock,-8.844505e+10,8.844505e+10
224,GarageQual_None,7.930425e+10,7.930425e+10
229,GarageCond_None,7.930320e+10,7.930320e+10
166,BsmtCond_None,6.149525e+10,6.149525e+10
218,GarageType_None,-5.136464e+10,5.136464e+10


In [47]:
target = "SalesPrice"

y = df[target]
y = np.log(y)
X = df.drop(columns=target)

X_dummy = pd.get_dummies(X, drop_first=True)


X_train, X_test, y_train, y_test = model_selection.train_test_split(X_dummy.values, y
                                        , test_size = 0.3, random_state = 126453)

pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha = 0.005, random_state = 1))
])

pipe.fit(X_train, y_train)

est = pipe.steps[-1][-1]

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("r2 score on train", metrics.r2_score(y_train, y_train_pred))
print("r2 score on test", metrics.r2_score(y_test, y_test_pred))

print("rmse on train", np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("rmse score on test", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

summary = pd.DataFrame({"feature": X_dummy.columns, "coeffient": est.coef_})
summary["mag"] = np.abs(summary.coeffient)
summary = summary.sort_values("mag", ascending = False)
summary = summary[summary.mag>0]
print("No of non-zero feature coefficients: ", summary.shape[0], "out of ", X_train.shape[1], )
summary

r2 score on train 0.927436179007304
r2 score on test 0.8081237487212176
rmse on train 0.10835615217914527
rmse score on test 0.1711980852768131
No of non-zero feature coefficients:  89 out of  258


,feature,coeffient,mag
15,GrLivArea,1.169254e-01,1.169254e-01
3,OverallQual,9.336155e-02,9.336155e-02
11,TotalBsmtSF,4.247913e-02,4.247913e-02
4,OverallCond,3.896623e-02,3.896623e-02
5,YearBuilt,3.410017e-02,3.410017e-02
90,Condition2_PosN,-3.064118e-02,3.064118e-02
26,GarageArea,2.962413e-02,2.962413e-02
8,BsmtFinSF1,2.721558e-02,2.721558e-02
250,SaleType_New,2.618441e-02,2.618441e-02
6,YearRemodAdd,2.287444e-02,2.287444e-02


In [36]:
target = "SalesPrice"

y = df[target]
y = np.log(y)
X = df.drop(columns=target)

X_dummy = pd.get_dummies(X, drop_first=True)


X_train, X_test, y_train, y_test = model_selection.train_test_split(X_dummy.values, y
                                        , test_size = 0.3, random_state = 1)

pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Ridge(alpha = 20.0, random_state = 1))
])

pipe.fit(X_train, y_train)

est = pipe.steps[-1][-1]

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("r2 score on train", metrics.r2_score(y_train, y_train_pred))
print("r2 score on test", metrics.r2_score(y_test, y_test_pred))

print("rmse on train", np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("rmse score on test", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

summary = pd.DataFrame({"feature": X_dummy.columns, "coeffient": est.coef_})
summary["mag"] = np.abs(summary.coeffient)
summary = summary.sort_values("mag", ascending = False)
summary = summary[summary.mag>0]
print("No of non-zero feature coefficients: ", summary.shape[0]
      , "out of ", X_train.shape[1], )
summary

r2 score on train 0.9422562432501826
r2 score on test 0.8756737399402362
rmse on train 0.0919903782017158
rmse score on test 0.15328521373355525
No of non-zero feature coefficients:  252 out of  258


,feature,coeffient,mag
110,RoofMatl_CompShg,0.093301,0.093301
114,RoofMatl_Tar&Grv,0.062909,0.062909
3,OverallQual,0.061513,0.061513
15,GrLivArea,0.059623,0.059623
32,PoolArea,0.059227,0.059227
116,RoofMatl_WdShngl,0.049640,0.049640
236,PoolQC_None,0.045813,0.045813
12,1stFlrSF,0.038872,0.038872
13,2ndFlrSF,0.036688,0.036688
4,OverallCond,0.033106,0.033106


In [45]:
target = "SalesPrice"

y = df[target]
y = np.log(y)
X = df.drop(columns=target)

X_dummy = pd.get_dummies(X, drop_first=True)


X_train, X_test, y_train, y_test = model_selection.train_test_split(X_dummy.values, y
                                        , test_size = 0.3, random_state = 453)

pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.ElasticNet(alpha = 0.01, random_state = 241, l1_ratio = 0.5))
])

pipe.fit(X_train, y_train)

est = pipe.steps[-1][-1]

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

print("r2 score on train", metrics.r2_score(y_train, y_train_pred))
print("r2 score on test", metrics.r2_score(y_test, y_test_pred))

print("rmse on train", np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("rmse score on test", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

summary = pd.DataFrame({"feature": X_dummy.columns, "coeffient": est.coef_})
summary["mag"] = np.abs(summary.coeffient)
summary = summary.sort_values("mag", ascending = False)
summary = summary[summary.mag>0]
print("No of non-zero feature coefficients: ", summary.shape[0]
      , "out of ", X_train.shape[1], )
summary

r2 score on train 0.9103587375532634
r2 score on test 0.8805768848102524
rmse on train 0.11526414423159057
rmse score on test 0.14882797402524284
No of non-zero feature coefficients:  91 out of  258


,feature,coeffient,mag
15,GrLivArea,0.109955,0.109955
3,OverallQual,0.089345,0.089345
25,GarageCars,0.044441,0.044441
5,YearBuilt,0.033467,0.033467
4,OverallCond,0.031952,0.031952
70,Neighborhood_NridgHt,0.024923,0.024923
38,MSZoning_RL,0.021759,0.021759
16,BsmtFullBath,0.019107,0.019107
60,Neighborhood_Crawfor,0.018960,0.018960
210,FireplaceQu_None,-0.017422,0.017422


In [65]:
target = "SalesPrice"

y = df[target]
y = np.log(y)
X = df.drop(columns=target)

X_dummy = pd.get_dummies(X, drop_first=True)

pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha = 0.01, random_state = 241))
])
scores = np.sqrt(- model_selection.cross_val_score(pipe, X_dummy.values, y, cv = 5
                                        , scoring="neg_mean_squared_error"))
print("rmse scores", scores, np.mean(scores))

rmse scores [0.12036647 0.15682778 0.14852173 0.12548199 0.19620989] 0.14948157314931704


In [60]:
metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'accuracy', 'roc_auc', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'brier_score_loss', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted'])

In [66]:
np.linspace(0.005, 0.02, 10)

array([0.005     , 0.00666667, 0.00833333, 0.01      , 0.01166667,
       0.01333333, 0.015     , 0.01666667, 0.01833333, 0.02      ])

In [78]:
%%time 

target = "SalesPrice"

y = df[target]
y = np.log(y)
X = df.drop(columns=target)

X_dummy = pd.get_dummies(X, drop_first=True)

pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=2, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.ElasticNet(random_state = 241, max_iter = 1000))
])

param_grid = {
    "est__alpha": np.linspace(0.005, 0.02, 10),
    "est__l1_ratio": np.linspace(0.1, 0.99, 10)
}
gsearch = model_selection.GridSearchCV(pipe, param_grid
                , scoring="neg_mean_squared_error", cv = 5)

gsearch.fit(X_dummy.values, y)

CPU times: user 54.1 s, sys: 222 ms, total: 54.3 s
Wall time: 9.13 s


In [74]:
gsearch.best_params_

{'est__alpha': 0.02, 'est__l1_ratio': 0.1988888888888889}

In [76]:
np.sqrt(-gsearch.best_score_)

0.14613046711164795

In [77]:
gsearch.best_estimator_

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', ElasticNet(alpha=0.02, copy_X=True, fit_intercept=True,
      l1_ratio=0.1988888888888889, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=241,
      selection='cyclic', tol=0.0001, warm_start=False))])

In [79]:
import pickle

In [80]:
with open("model.pkl", "wb") as f:
    pickle.dump(gsearch, f)

In [81]:
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

422    11.635143
121    11.512925
797    11.608236
504    11.898188
675    11.908340
Name: SalesPrice, dtype: float64

In [87]:
pd.DataFrame({"actual": y_test[:5], "prediction": model.predict(X_test)[:5]})

,actual,prediction
422,11.635143,11.641225
121,11.512925,11.416992
797,11.608236,11.554195
504,11.898188,11.905789
675,11.908340,11.903607
